# Part 1: Paper Selection and Analysis Preparation

Now that we have a database table of all the papers, we can apply exclusion criteria and analyze them. For this, we can make auxiliary tables.

**NOTE:** There is no one right way to go about documenting paper selection. Most use Excel, which is an amazing tool, especially for collaborative work!

This notebook and the part-1b Python module allow for storage of information in a queryable database and prompted display of one article at a time in the command line, but is not necessarily better or faster than alternatives. It's all up to personal preference.

In [2]:
__author__ = "Christine Mendoza, \
    with some sqlite3-related code from Dr. Gary Bishop's / Dr. John Majikes' UNC Chapel Hill COMP421 (Databases) class"

Before going further, make sure to define your file paths.

In [3]:
DB_PATH: str = "./data/sqlite/analysis.db"

## Create Tables

Set up your database connection. A file with the path specified by `OUTPUT_DB` will automatically be created if one does not exist.

In [4]:
import sqlite3
db = sqlite3.Connection(DB_PATH)
cursor = db.cursor()

Edit the code below to define tables for each stage with the appropriate exclusion criteria. 

**Notes:** 
- Duplicates are identified separately.
- 0 and 1 denote false and true, respectively.
- I have chosen to allow for any number of columns in analysis to be true and thus do not have columns such as 'both' or 'all_of_the_above', except to denote universal design in `Disability_of_Target_Population`. (Also note that 'all' is a reserved keyword in SQL.)

In [5]:
create_tables = """
    CREATE TABLE Exclusion_Stage_1
        (paper_analysis_id INTEGER REFERENCES All_Papers(paper_analysis_id) NOT NULL,
        off_topic INTEGER CHECK(off_topic IN (0, 1)), -- Off-topic (0 is false, 1 is true)
        intro_proceedings INTEGER CHECK(intro_proceedings IN (0, 1))); -- Introductory paper to conference/workshop proceedings
    CREATE TABLE Exclusion_Stage_2
        (paper_analysis_id INTEGER REFERENCES All_Papers(paper_analysis_id) NOT NULL,
        preliminary INTEGER CHECK(preliminary IN (0, 1)), -- Preliminary work
        extended_abstract INTEGER CHECK(extended_abstract IN (0, 1)), -- Extended abstract
        requirement_analysis INTEGER CHECK(requirement_analysis IN (0, 1)), -- Analysis of requirements rather than implementation of application
        data_system INTEGER CHECK(data_system IN (0, 1)), -- System for data collection and storage
        conceptual INTEGER CHECK(conceptual IN (0, 1)), -- Conceptual model
        user_study INTEGER CHECK(user_study IN (0, 1)), -- User study or guidelines
        survey INTEGER CHECK(survey IN (0, 1))); -- Survey or review
    CREATE TABLE Duplicates
        (parent_id INTEGER REFERENCES All_Papers(paper_analysis_id) NOT NULL,
        child_id INTEGER REFERENCES All_Papers(paper_analysis_id) NOT NULL, -- The duplicate
        UNIQUE (parent_id, child_id));
    CREATE TABLE Device_Classification
        (paper_analysis_id INTEGER REFERENCES All_Papers(paper_analysis_id) NOT NULL,
        mobile INTEGER CHECK(mobile IN (0, 1)), -- Mobile application
        web INTEGER CHECK(web IN (0, 1)), -- Web application
        wearable INTEGER CHECK(wearable IN (0, 1)),
        smart INTEGER CHECK(smart IN (0, 1)),
        tactile_or_public INTEGER CHECK(tactile_or_public IN (0, 1))); -- Tactile / public interfaces
    CREATE TABLE Usage_Context
        (paper_analysis_id INTEGER REFERENCES All_Papers(paper_analysis_id) NOT NULL,
        indoor INTEGER CHECK(indoor IN (0, 1)),
        outdoor INTEGER CHECK(outdoor IN (0, 1)));
    CREATE TABLE Disability_of_Target_Population
        (paper_analysis_id INTEGER REFERENCES All_Papers(paper_analysis_id) NOT NULL,
        visual INTEGER CHECK(visual IN (0, 1)), -- Specifically mentioned/considered (and so on with other columns)
        mobility INTEGER CHECK(mobility IN (0, 1)),
        cognitive INTEGER CHECK(cognitive IN (0, 1)),
        hearing INTEGER CHECK(hearing IN (0, 1)),
        other INTEGER CHECK(other IN (0, 1)),
        universal INTEGER CHECK(universal IN (0, 1))); -- Universal design
    CREATE TABLE Technology_Type
        (paper_analysis_id INTEGER REFERENCES All_Papers(paper_analysis_id) NOT NULL,
        environment INTEGER CHECK(environment IN (0, 1)), -- Environment representation and modeling
        localization INTEGER CHECK(localization IN (0, 1)),
        wayfinding INTEGER CHECK(wayfinding IN (0, 1))); -- Wayfinding and navigation
    CREATE TABLE Data_Sources
        (paper_analysis_id INTEGER REFERENCES All_Papers(paper_analysis_id) NOT NULL,
        crowdsourcing INTEGER CHECK(crowdsourcing IN (0, 1)),
        crowdsensing INTEGER CHECK(crowdsensing IN (0, 1)), -- Crowdsensing and pervasive sensing
        official INTEGER CHECK(official IN (0, 1)), -- Official and expert
        other INTEGER CHECK(other IN (0, 1)));
    CREATE TABLE User_Involvement
        (paper_analysis_id INTEGER REFERENCES All_Papers(paper_analysis_id) NOT NULL,
        design INTEGER CHECK(design IN (0, 1)), -- Involved in design time
        evaluation INTEGER CHECK(evaluation IN (0, 1))); -- Involved in system evaluation
    """

Finally, we can create the table(s)!

In [6]:
cursor.executescript(create_tables)

Check that the table(s) was/were actually created.

In [7]:
cursor.execute('''
               SELECT name
               FROM sqlite_master
               WHERE type='table' AND
                     name NOT LIKE 'sqlite_%' ''')
table_names = cursor.fetchall()

# List table names
print("Here are the tables in the database:")
for name in table_names:
    print(name[0])

Here are the tables in the database:
All_Papers
Exclusion_Stage_1
Exclusion_Stage_2
Duplicates
Device_Classification
Usage_Context
Disability_of_Target_Population
Technology_Type
Data_Sources
User_Involvement


Make sure to commit your changes!

In [8]:
db.commit()

Now we can move on to the actual paper selection via exclusion criteria (part 2)!